1. Filter the data and create a new DataFrame or table to retrieve all the rows where the total_votes count is equal to or greater than 20 to pick reviews that are more likely to be helpful and to avoid having division by zero errors later on.

2. Filter the new DataFrame or table created in Step 1 and create a new DataFrame or table to retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%.

3. If you use the SQL option below, you’ll need to cast your columns as floats using WHERE CAST(helpful_votes AS FLOAT)/CAST(total_votes AS FLOAT) >=0.5.
Filter the DataFrame or table created in Step 2, and create a new DataFrame or table that retrieves all the rows where a review was written as part of the Vine program (paid), vine == 'Y'.

4. Repeat Step 3, but this time retrieve all the rows where the review was not part of the Vine program (unpaid), vine == 'N'.

5. Determine the total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews for the two types of review (paid vs unpaid).

In [1]:
import os
spark_version = 'spark-3.3.1'
os.environ['SPARK_VERSION'] = spark_version

# install Java and spark
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

#  Postgres driver that will allow Spark to interact with Postgres
!wget https://jdbc.postgresql.org/download/postgresql-42.2.17.jar

#  start a Spark session with an additional option that adds the driver to Spark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Vine_review").config("spark.driver.extraClassPath","/content/postgresql-42.2.17.jar").getOrCreate()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu b

In [2]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://module17awsbucket2023.s3.us-west-2.amazonaws.com/vine_table.csv"
spark.sparkContext.addFile(url)
vine_df = spark.read.csv(SparkFiles.get("vine_table.csv"), sep=",", header=True, inferSchema=True)
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RMDCHWD0Y5OZ9|          3|            0|          1|   N|                N|
| RZSL0BALIYUNU|          5|            0|          0|   N|                Y|
| RIZR67JKUDBI0|          3|            0|          1|   N|                Y|
|R27HL570VNL85F|          5|            0|          0|   N|                Y|
|R34EBU9QDWJ1GD|          5|            0|          0|   N|                Y|
|R1WCUI4Z1SIQEO|          5|            0|          0|   N|                N|
| RL5LNO26GAVJ1|          2|            3|          4|   N|                Y|
|R3GYQ5W8JHP8SB|          5|            0|          0|   N|                Y|
|R30SHYQXGG5EYC|          5|            0|          0|   N|                Y|
|R14YLXA56NP51I|          5|            1|          1|   N|     

In [23]:
vine_df.describe()

DataFrame[summary: string, review_id: string, star_rating: string, helpful_votes: string, total_votes: string, vine: string, verified_purchase: string]

1. Filter the data and create a new DataFrame or table to retrieve all the rows where the total_votes count is equal to or greater than 20 to pick reviews that are more likely to be helpful and to avoid having division by zero errors later on.

In [3]:
# df where total_votes >= 20 (avoid /0)
voted_reviews = vine_df.filter("total_votes>=20")
voted_reviews.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2243Y3OD8U6KQ|          5|           47|         61|   N|                N|
|R2TGT0CDTCAAHW|          5|           21|         23|   N|                Y|
| RX4D22YSXEF4P|          1|           37|         38|   N|                Y|
|R3FL2NTLFUSPTQ|          5|           33|         37|   N|                N|
|R3QTP3YNZXAPPF|          3|           23|         24|   N|                Y|
|R36V6V42VN5AS5|          5|           34|         37|   N|                Y|
|R27LZWE27BJPOB|          5|           22|         23|   N|                N|
|  RMRD6SMF2AUQ|          3|            4|         21|   N|                N|
| RMPCXKWX3T57Y|          1|            1|         73|   N|                N|
|R1P7GJ0IN2BRNH|          5|           37|         37|   N|     

2. Filter the new DataFrame or table created in Step 1 and create a new DataFrame or table to retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%.


In [4]:
# Create new dataframe where at least 50% of the voters thought the review was helpful
# helpful_votes:x::total_votes:100 ; x>=50
helpful_reviews = voted_reviews.filter('((helpful_votes*100)/total_votes)>=(50)')
helpful_reviews.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2243Y3OD8U6KQ|          5|           47|         61|   N|                N|
|R2TGT0CDTCAAHW|          5|           21|         23|   N|                Y|
| RX4D22YSXEF4P|          1|           37|         38|   N|                Y|
|R3FL2NTLFUSPTQ|          5|           33|         37|   N|                N|
|R3QTP3YNZXAPPF|          3|           23|         24|   N|                Y|
|R36V6V42VN5AS5|          5|           34|         37|   N|                Y|
|R27LZWE27BJPOB|          5|           22|         23|   N|                N|
|R1P7GJ0IN2BRNH|          5|           37|         37|   N|                Y|
|R2R6JPF9KOD2HJ|          5|           19|         20|   N|                Y|
|R2J0ZZGFXKM8KR|          2|           21|         22|   N|     

3. If you use the SQL option below, you’ll need to cast your columns as floats using WHERE CAST(helpful_votes AS FLOAT)/CAST(total_votes AS FLOAT) >=0.5.
Filter the DataFrame or table created in Step 2, and create a new DataFrame or table that retrieves all the rows where a review was written as part of the Vine program (paid), vine == 'Y'.

In [5]:
# Create dataframe of only Vine program participants
vine_members = helpful_reviews.filter(helpful_reviews["vine"] == "Y")
vine_members.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1R9RU7JW0MFR2|          4|           20|         23|   Y|                N|
|R19EFYNN3W8Q07|          5|           26|         32|   Y|                N|
|R34DJ1R8AEU0SG|          5|           29|         35|   Y|                N|
|R25P5CXK5L9RHF|          5|          146|        161|   Y|                N|
|R2E9VZB3I4LSN5|          5|           55|         59|   Y|                N|
| RKYLHZL7EPELX|          4|           19|         25|   Y|                N|
|R1U13EKGQD3ZE6|          5|           22|         25|   Y|                N|
| RYW05F1MUEF01|          5|           87|        102|   Y|                N|
|R2SW4NXNO7HZJ5|          4|           28|         33|   Y|                N|
|R2016NFLSUR97Y|          2|           26|         37|   Y|     

4. Repeat Step 3, but this time retrieve all the rows where the review was not part of the Vine program (unpaid), vine == 'N'

In [6]:
# Create dataframe of only NOT part of the vine program
non_vine_members = helpful_reviews.filter(helpful_reviews["vine"] == "N")
non_vine_members.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2243Y3OD8U6KQ|          5|           47|         61|   N|                N|
|R2TGT0CDTCAAHW|          5|           21|         23|   N|                Y|
| RX4D22YSXEF4P|          1|           37|         38|   N|                Y|
|R3FL2NTLFUSPTQ|          5|           33|         37|   N|                N|
|R3QTP3YNZXAPPF|          3|           23|         24|   N|                Y|
|R36V6V42VN5AS5|          5|           34|         37|   N|                Y|
|R27LZWE27BJPOB|          5|           22|         23|   N|                N|
|R1P7GJ0IN2BRNH|          5|           37|         37|   N|                Y|
|R2R6JPF9KOD2HJ|          5|           19|         20|   N|                Y|
|R2J0ZZGFXKM8KR|          2|           21|         22|   N|     

5. Determine the total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews for the two types of review (paid vs unpaid).

Part of Vine

In [33]:
groupped_vine_members = vine_members.groupBy("star_rating").agg({"star_rating":"count"})
groupped_vine_members.show()

+-----------+------------------+
|star_rating|count(star_rating)|
+-----------+------------------+
|          3|                 9|
|          5|                34|
|          4|                16|
|          2|                 1|
+-----------+------------------+



In [46]:
total_reviews = groupped_vine_members.agg({'count(star_rating)': 'sum'}).show()

+-----------------------+
|sum(count(star_rating))|
+-----------------------+
|                     60|
+-----------------------+



In [10]:
groupped_vine_members.describe().show()

+-------+------------------+------------------+
|summary|       star_rating|count(star_rating)|
+-------+------------------+------------------+
|  count|                 4|                 4|
|   mean|               3.5|              15.0|
| stddev|1.2909944487358056|14.071247279470288|
|    min|                 2|                 1|
|    max|                 5|                34|
+-------+------------------+------------------+



In [18]:
vine_percentage = groupped_vine_members.withColumn('percentage',groupped_vine_members['count(star_rating)']*100/60)
vine_percentage.orderBy('star_rating').show()

+-----------+------------------+------------------+
|star_rating|count(star_rating)|        percentage|
+-----------+------------------+------------------+
|          2|                 1|1.6666666666666667|
|          3|                 9|              15.0|
|          4|                16|26.666666666666668|
|          5|                34|56.666666666666664|
+-----------+------------------+------------------+



Not Part of Vine

In [8]:
groupped_non_vine_members = non_vine_members.groupBy("star_rating").agg({"star_rating":"count"})
groupped_non_vine_members.show()

+-----------+------------------+
|star_rating|count(star_rating)|
+-----------+------------------+
|          1|              1532|
|          3|              1292|
|          5|              8212|
|          4|              2687|
|          2|               754|
+-----------+------------------+



In [49]:
total_reviews_non = groupped_non_vine_members.agg({'count(star_rating)': 'sum'}).show()

+-----------------------+
|sum(count(star_rating))|
+-----------------------+
|                  14477|
+-----------------------+



In [20]:
groupped_non_vine_members.describe().show()

+-------+------------------+------------------+
|summary|       star_rating|count(star_rating)|
+-------+------------------+------------------+
|  count|                 5|                 5|
|   mean|               3.0|            2895.4|
| stddev|1.5811388300841898| 3054.709937129874|
|    min|                 1|               754|
|    max|                 5|              8212|
+-------+------------------+------------------+



In [22]:
not_vine_percentage = groupped_non_vine_members.withColumn('percentage',groupped_non_vine_members['count(star_rating)']*100/14477)
not_vine_percentage.orderBy('star_rating').show()

+-----------+------------------+------------------+
|star_rating|count(star_rating)|        percentage|
+-----------+------------------+------------------+
|          1|              1532| 10.58230296332113|
|          2|               754| 5.208261380120191|
|          3|              1292| 8.924500932513642|
|          4|              2687|18.560475236582164|
|          5|              8212| 56.72445948746287|
+-----------+------------------+------------------+

